# Arithmetic

An important function of pandas is the arithmetic behaviour for objects with different indices. When adding objects, if the index pairs are not equal, the corresponding index in the result will be the union of the index pairs. For users with database experience, this is comparable to an automatic [outer join](https://en.wikipedia.org/wiki/Join_(SQL)#Outer_join) on the index labels. Let’s look at an example:

In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()
s1 = pd.Series(rng.normal(size=5))
s2 = pd.Series(rng.normal(size=7))

If you add these values, you get:

In [2]:
s1 + s2

0    0.452985
1   -2.751479
2    1.170106
3   -1.583008
4   -0.507949
5         NaN
6         NaN
dtype: float64

The internal data matching leads to missing values at the points of the labels that do not overlap. Missing values are then passed on in further arithmetic calculations.

For DataFrames, alignment is performed for both rows and columns:

In [3]:
df1 = pd.DataFrame(rng.normal(size=(5,3)))
df2 = pd.DataFrame(rng.normal(size=(7,2)))

When the two DataFrames are added together, the result is a DataFrame whose index and columns are the unions of those in each of the DataFrames above:

In [4]:
df1 + df2

,0,1,2
0,-1.740413,0.566677,NaN
1,1.992954,-2.637117,NaN
2,-0.111516,-0.340413,NaN
3,0.418716,-0.847758,NaN
4,0.625907,1.675521,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


Since column 2 does not appear in both DataFrame objects, its values appear as missing in the result. The same applies to the rows whose labels do not appear in both objects.

## Arithmetic methods with fill values

In arithmetic operations between differently indexed objects, a special value (e.g. `0`) can be useful if an axis label is found in one object but not in the other.  The `add` method can pass the `fill_value` argument:

In [5]:
df12 = df1.add(df2, fill_value=0)

df12

,0,1,2
0,-1.740413,0.566677,0.715587
1,1.992954,-2.637117,0.888983
2,-0.111516,-0.340413,0.811355
3,0.418716,-0.847758,-0.233179
4,0.625907,1.675521,0.494883
5,1.780402,-0.349901,NaN
6,0.922128,-0.487242,NaN


In the following example, we set the two remaining NaN values to `0`:

In [6]:
df12.iloc[[5,6], [2]] = 0

In [7]:
df12

,0,1,2
0,-1.740413,0.566677,0.715587
1,1.992954,-2.637117,0.888983
2,-0.111516,-0.340413,0.811355
3,0.418716,-0.847758,-0.233179
4,0.625907,1.675521,0.494883
5,1.780402,-0.349901,0.000000
6,0.922128,-0.487242,0.000000


## Arithmetic methods

Method | Description
:----- | :----------
`add`, `radd` | methods for addition (`+`)
`sub`, `rsub` | methods for subtraction (`-`)
`div`, `rdiv` | methods for division (`/`)
`floordiv`, `rfloordiv` | methods for floor division (`//`)
`mul`, `rmul` | methods for multiplication (`*`)
`pow`, `rpow` | methods for exponentiation (`**`)

`r` (English: _reverse_) reverses the method.

## Operations between DataFrame and Series

As with NumPy arrays of different dimensions, the arithmetic between DataFrame and Series is also defined.

In [8]:
s1 + df12

,0,1,2,3,4
0,-2.533022,-1.635689,2.219348,NaN,NaN
1,1.200346,-4.839483,2.392744,NaN,NaN
2,-0.904124,-2.542779,2.315117,NaN,NaN
3,-0.373892,-3.050124,1.270582,NaN,NaN
4,-0.166702,-0.526845,1.998644,NaN,NaN
5,0.987793,-2.552267,1.503761,NaN,NaN
6,0.129520,-2.689608,1.503761,NaN,NaN


If we add `s1` with `df12`, the addition is done once for each line. This is called _broadcasting_. By default, the arithmetic between the DataFrame and the series corresponds to the index of the series in the columns of the DataFrame, with the rows being broadcast down.

If an index value is found neither in the columns of the DataFrame nor in the index of the series, the objects are re-indexed to form the union:

If instead you want to transfer the columns and match the rows, you must use one of the arithmetic methods, for example:

In [9]:
df12.add(s2, axis='index')

,0,1,2
0,-0.494820,1.812271,1.961180
1,1.443841,-3.186230,0.339870
2,-0.445171,-0.674068,0.477700
3,-0.380913,-1.647387,-1.032809
4,0.260930,1.310545,0.129906
5,2.116184,-0.014119,0.335782
6,1.471784,0.062413,0.549655


The axis number you pass is the axis to be aligned to. In this case, the row index of the DataFrame (`axis='index'` or `axis=0`) is to be adjusted and transmitted.

## Function application and mapping

`numpy.ufunc` (element-wise array methods) also work with pandas objects:

In [10]:
np.abs(df12)

,0,1,2
0,1.740413,0.566677,0.715587
1,1.992954,2.637117,0.888983
2,0.111516,0.340413,0.811355
3,0.418716,0.847758,0.233179
4,0.625907,1.675521,0.494883
5,1.780402,0.349901,0.000000
6,0.922128,0.487242,0.000000


Another common operation is to apply a function to one-dimensional arrays on each column or row. The [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method does just that:

In [11]:
df12

,0,1,2
0,-1.740413,0.566677,0.715587
1,1.992954,-2.637117,0.888983
2,-0.111516,-0.340413,0.811355
3,0.418716,-0.847758,-0.233179
4,0.625907,1.675521,0.494883
5,1.780402,-0.349901,0.000000
6,0.922128,-0.487242,0.000000


In [12]:
f = lambda x: x.max() - x.min()

df12.apply(f)

0    3.733368
1    4.312639
2    1.122163
dtype: float64

Here the function `f`, which calculates the difference between the maximum and minimum of a row, is called once for each column of the frame. The result is a row with the columns of the frame as index.

If you pass `axis='columns'` to `apply`, the function will be called once per line instead:

In [13]:
df12.apply(f, axis='columns')

0    2.456000
1    4.630072
2    1.151768
3    1.266474
4    1.180639
5    2.130302
6    1.409370
dtype: float64

Many of the most common array statistics (such as `sum` and `mean`) are DataFrame methods, so the use of `apply` is not necessary.

The function passed to apply does not have to return a single value; it can also return a series with multiple values:

In [14]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

df12.apply(f)

,0,1,2
min,-1.740413,-2.637117,-0.233179
max,1.992954,1.675521,0.888983


You can also use element-wise Python functions. Suppose you want to round each floating point value in `df12` to two decimal places, you can do this with [pandas.DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html):

In [15]:
f = lambda x: round(x, 2)

df12.applymap(f)

,0,1,2
0,-1.74,0.57,0.72
1,1.99,-2.64,0.89
2,-0.11,-0.34,0.81
3,0.42,-0.85,-0.23
4,0.63,1.68,0.49
5,1.78,-0.35,0.00
6,0.92,-0.49,0.00


The reason for the name `applymap` is that Series has a `map` method for applying an element-wise function:

In [16]:
df12[2].map(f)

0    0.72
1    0.89
2    0.81
3   -0.23
4    0.49
5    0.00
6    0.00
Name: 2, dtype: float64